This script is based on: https://github.com/microsoft/AzureDataRetrievalAugmentedGenerationSamples/blob/main/Python/CosmosDB-NoSQL_VectorSearch/CosmosDB-NoSQL-Vector_AzureOpenAI_Tutorial.ipynb

# Create an Azure Cosmos DB for NoSQL resource

Let's start by creating an Azure Cosmos DB for NoSQL Resource (Cosmos DB Account) by following [this section in the Quickstart guide](https://learn.microsoft.com/en-us/azure/cosmos-db/nosql/quickstart-portal#create-account)

## Get Cosmos DB Account Key and Endpoint
Once the account is provisioned, head over to the provisioned account and navigate to **"Settings > Keys"** section in the left-side panel. From the Keys section, make a note of the **Primary Key and the URI** - these will be used later to connect to the cosmos DB account through the python client.
Store the Primary Key and URI in a .env file

# Provision Azure Open AI resource
Finally, let's setup our Azure OpenAI resource Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at [https://aka.ms/oai/access](https://aka.ms/oai/access)

Once you have access, complete the following step:
1. Create an Azure OpenAI resource [following this quickstart](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=-eb-portal)
2. Deploy an embeddings model. For more information on embeddings, refer to [this article](https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings)
3. Deploy a completions model. For more information on completions, refer to [this article](https://learn.microsoft.com/azure/ai-services/openai/how-to/completions)
4. Make a note of the endpoint and key for your Azure OpenAI resource
5. Make a note of the **deployment names** of the embedding and completion models.

Store the Endpoint, Key, and deployment names in the .env file


# Install the required libraries

In [ ]:
'''
! pip install numpy
! pip install openai
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
'''

# Necessary imports

In [1]:
import json
import datetime
import time
import urllib 

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential

#Cosmos DB imports
from azure.cosmos import CosmosClient
from azure.cosmos.aio import CosmosClient as CosmosAsyncClient
from azure.cosmos import PartitionKey, exceptions

from openai import AzureOpenAI
from dotenv import load_dotenv

# Load Keys, Endpoints, and other variables from the .env file

In [ ]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "localsettings.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

OPENAI_API_KEY = config['openai_api_key']
OPENAI_API_ENDPOINT = config['openai_api_endpoint']
OPENAI_API_VERSION = config['openai_api_version'] # at the time of authoring, the api version is 2024-02-01
COMPLETIONS_MODEL_DEPLOYMENT_NAME = config['completions_model_deployment_name']
EMBEDDING_MODEL_DEPLOYMENT_NAME = config['embedding_model_deployment_name']
COSMOSDB_NOSQL_ACCOUNT_KEY = config['cosmosdb_nosql_account_key']
COSMOSDB_NOSQL_ACCOUNT_ENDPOINT = config['cosmosdb_nosql_account_endpoint']

print("OPENAI_API_ENDPOINT: ", OPENAI_API_ENDPOINT)
print("COSMOSDB_NOSQL_ACCOUNT_ENDPOINT: ", COSMOSDB_NOSQL_ACCOUNT_ENDPOINT)

# Instantiate the Azure Open AI client

In [9]:
AOAI_client = AzureOpenAI(api_key=OPENAI_API_KEY, azure_endpoint=OPENAI_API_ENDPOINT, api_version=OPENAI_API_VERSION,)

# Generating Embedding
We'll use the deployed embeddings model to generate the embeddings

In [10]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = AOAI_client.embeddings.create(input=text, model=EMBEDDING_MODEL_DEPLOYMENT_NAME)
    embeddings =response.model_dump()
    time.sleep(0.5) 
    return embeddings['data'][0]['embedding']

In [11]:
print("Embeddings generated: ", generate_embeddings("Today is DiskANN day"))

Embeddings generated:  [-0.011703060939908028, -0.0012448977213352919, 0.007479604799300432, -0.011437391862273216, -0.0003205909742973745, 0.016634967178106308, -0.03264322876930237, -0.0022445626091212034, -0.01636248640716076, -0.03250698745250702, 0.002656690077856183, 0.033569663763046265, -0.0007425108342431486, 0.0001243194710696116, -0.020286213606595993, 0.015231690369546413, 0.025109129026532173, -0.0018562771147117019, 0.01609000563621521, -0.002219017595052719, 0.0003782803250942379, 0.010933301411569118, -0.01824260503053665, -0.007949634455144405, 0.014863841235637665, -0.018283478915691376, 0.01625349372625351, -0.027684073895215988, 0.0026600961573421955, 0.008065438829362392, 0.005350846331566572, -0.030381636694073677, 0.02132164128124714, -0.015776652842760086, -0.031117334961891174, -0.024455172941088676, 0.0018562771147117019, 0.0012883244780823588, 0.016880201175808907, 0.0035729077644646168, -0.0038317646831274033, 0.007295679766684771, 0.003777268575504422, -0.0

# Load the data with embeddings or generate embeddings
We have a sample data file with embeddings but you can generate the embeddings afresh before uploading the data.

In [2]:
data_file = open(file="./Datasets/netflix_titles.json", mode="r")

data = json.load(data_file)
data_file.close()

In [ ]:
# Take a peek at one data item
print(json.dumps(data[0], indent=2))

{
  "id": "s2",
  "type": "TV Show",
  "title": "Blood & Water",
  "director": "",
  "cast": "Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",
  "country": "South Africa",
  "date_added": "September 24, 2021",
  "release_year": "2021",
  "rating": "TV-MA",
  "duration": "2 Seasons",
  "description": "After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.",
  "listed_in": "International TV Shows, TV Dramas, TV Mysteries"
}


In [15]:
# Generate embeddings for title and content fields
n = 0
for item in data:
    n+=1
    #print(item['title'] + ' | ' + item['description'])
    doc_embeddings = generate_embeddings(item['title'] + ' | ' + item['description'])
    item['docVector'] = doc_embeddings
    print("Creating embeddings for item:", n, "/" ,len(data), end='\r')

In [ ]:
# If you have the embeddings pre-computed, you can load them from the file
# data_file = open(file="./Datasets/netflix_titles_withembeddings.json", mode="r")

# data = json.load(data_file)
# data_file.close()

In [5]:
print(len(data))
#print(json.dumps(data[0], indent=2))

8807


# Connect and setup Cosmos DB for NoSQL
Now that we have the data with embeddings ready, we need to upload this data to Azure Cosmos DB container with vector search capability. For this, we need to create a new container (as vector search is currently supported in new containers only) with vector embedding and indexing policy.

## Set up the connection

In [12]:
cosmos_client = CosmosClient(url=COSMOSDB_NOSQL_ACCOUNT_ENDPOINT, credential=COSMOSDB_NOSQL_ACCOUNT_KEY)

## Create a new database or use existing one

In [19]:
#create database
DATABASE_NAME = "vector-nosql-db"
db= cosmos_client.create_database_if_not_exists(
    id=DATABASE_NAME
)
properties = db.read()
print(json.dumps(properties))

{"id": "vector-nosql-db", "_rid": "7E1lAA==", "_self": "dbs/7E1lAA==/", "_etag": "\"00008c01-0000-0500-0000-66703a980000\"", "_colls": "colls/", "_users": "users/", "_ts": 1718631064}


## Author the vector embedding policy
Vector embedding policy defines the necessary information for the vector search queries as detailed below: 
* “path”: what properties contain vectors 
* “datatype”: What type are the vector’s elements (default Float32) 
* “dimensions”: The length of each vector in the path (default 1536) 
* “distanceFunction”: The metric used to compute distance/similarity (default Cosine)

In [20]:
vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path":"/docVector",
            "dataType":"float32",
            "distanceFunction":"cosine",
            "dimensions":1536
        }
    ]
}

## Add vector indexes to indexing policy

In [21]:
indexing_policy = {
    "includedPaths": [
        {
            "path": "/*"
        }
    ],
    "excludedPaths": [
        {
            "path": "/docVector/?"
        },
        {
            "path": "/\"_etag\"/?"
        }
    ],
    "vectorIndexes": [
        {"path": "/docVector",
         "type": "diskANN"
        }
    ]
}


## Create container with the embedding and indexing policy

In [22]:
CONTAINER_NAME = "vector-diskann"
try:    
    container = db.create_container_if_not_exists(
                    id=CONTAINER_NAME,
                    partition_key=PartitionKey(path='/partKey', kind='Hash'),
                    indexing_policy=indexing_policy,
                    vector_embedding_policy=vector_embedding_policy)

    print('Container with id \'{0}\' created'.format(id))

except exceptions.CosmosResourceExistsError:
    print('A container with id \'{0}\' already exists'.format(id))

Container with id '<built-in function id>' created


In [14]:
# Skip creating the container if it already exists
DATABASE_NAME = "vector-nosql-db"
CONTAINER_NAME = "vector-diskann"
db= cosmos_client.get_database_client(DATABASE_NAME)
container = db.get_container_client(CONTAINER_NAME)

## Upload data to the container
Azure Cosmos DB Python SDK does not currently support bulk inserts so we'll have to insert the items sequentially

In [ ]:
# with open('./DataSet/AzureServices/text-sample_w_embeddings.json') as f:
#    data = json.load(f)

container_client = db.get_container_client(CONTAINER_NAME)

for item in data:
    item['partKey'] = item['release_year']
    container_client.upsert_item(item)
    print("writing item {} - {}".format(item['id'], container_client.client_connection.last_response_headers['x-ms-request-charge']))

## Vector search in Azure Cosmos DB for NoSQL
Let's write a function that will take in user's query, generate embeddings for the query text and then use the embedding to run a vector search to find the similar items. The most similar items must be used as additional knowledgebase for the completions model to answer the user's query

In [8]:
print("Embeddings generated: ", generate_embeddings("Best romantic comedy movies to watch in family with kids coming to tenage years"))

Embeddings generated:  [0.009276620112359524, -0.03581053391098976, -0.010156014934182167, -0.006459913216531277, 0.00831126980483532, 0.022811362519860268, -0.0051342095248401165, -0.019280560314655304, -0.022414643317461014, -0.005663168616592884, -0.0013827321818098426, 0.013713264837861061, -0.0052763670682907104, 0.0034217042848467827, 0.012741303071379662, 0.015392710454761982, 0.023287424817681313, -0.009184052236378193, 0.004922625608742237, -0.02835220843553543, 0.006578928790986538, -0.019267335534095764, 0.004568884149193764, -0.027902593836188316, 0.003157224738970399, 0.004254814703017473, 0.03649817779660225, -0.038164399564266205, -0.0015943158650770783, -0.017349859699606895, 0.030520940199494362, -0.012000760063529015, 0.0032249975483864546, -0.03284836187958717, -0.006364039145410061, -0.01992853544652462, -0.015458830632269382, -0.002059634542092681, 0.023631248623132706, -0.01356780156493187, 0.007471547462046146, -0.004598638508468866, -0.002352215116843581, -0.010

In [15]:
# Simple function to assist with vector search
def vector_search(query, num_results=5, printQuery=False):
    query_embedding = generate_embeddings(query)

    querystring = "SELECT TOP {} c.id, c.type, c.title, c.rating, c.release_year, c.description, VectorDistance(c.docVector,{}) AS similarityScore FROM c".format(num_results, query_embedding)
    
    results = container_client.query_items(
            query=querystring,
            enable_cross_partition_query=True)
    
    return results

In [18]:
# Simple function to assist with vector search
def vector_search_ordered(query, num_results=3, printQuery=False):
    query_embedding = generate_embeddings(query)

    querystring = "SELECT TOP {} c.id, c.type, c.title, c.rating, c.release_year, c.description, VectorDistance(c.docVector,{}) AS similarityScore FROM c ORDER BY VectorDistance(c.docVector,{})".format(num_results, query_embedding, query_embedding)
    
    if printQuery:
        print(querystring)

    results = container_client.query_items(
            query=querystring,
            enable_cross_partition_query=True)
    
    return results

Let's run a test below

In [20]:
DATABASE_NAME = "vector-nosql-db"
CONTAINER_NAME = "vector-diskann"
db= cosmos_client.get_database_client(DATABASE_NAME)
container_client = db.get_container_client(CONTAINER_NAME)

In [ ]:
query = "Best romantic comedy movies to watch in family with kids coming to tenage years"

results = vector_search(query)

# print
for result in results: 
    # print(result)
    print(f"Similarity Score: {result['similarityScore']}")
    print(f"Id: {result['id']}")  
    print(f"Type: {result['type']}")        
    print(f"Title: {result['title']}")  
    print(f"Rating: {result['rating']}")  
    print(f"Description: {result['description']}") 
    print(f"Release_year: {result['release_year']}\n") 

# print("Consumed RUs: {} ".format(container_client.client_connection.last_response_headers['x-ms-request-charge']))

In [28]:
# query = "What are some NoSQL databases in Azure?"
# query = "What are the services for event messaging patterns?"

#query = "Best romantic comedy movies to watch in family with kids coming to tenage years"
#query = "Action movie with zombies and a lot of blood"
query = "Sports movie that shows the struggle of a team to win the championship"

results = vector_search_ordered(query)

for result in results: 
    # print(result)
    print(f"Similarity Score: {result['similarityScore']}")
    print(f"Id: {result['id']}")  
    print(f"Type: {result['type']}")        
    print(f"Title: {result['title']}")  
    print(f"Rating: {result['rating']}")  
    print(f"Description: {result['description']}") 
    print(f"Release_year: {result['release_year']}\n") 

# print("Consumed RUs: {} ".format(container_client.client_connection.last_response_headers['x-ms-request-charge']))

Similarity Score: 0.862960131157242
Id: s5323
Type: Movie
Title: Undefeated
Rating: PG-13
Description: An inspirational profile of an inner-city high school football team's valiant effort to reach the school's first-ever playoff game.
Release_year: 2011

Similarity Score: 0.8627748584678387
Id: s8479
Type: Movie
Title: The Rebound
Rating: TV-MA
Description: This documentary follows three players on the Miami Heat Wheels, who face long odds on the way to the national wheelchair basketball championship.
Release_year: 2016

Similarity Score: 0.8565862992174109
Id: s5747
Type: Movie
Title: A Mighty Team
Rating: TV-MA
Description: When a fit of anger leads to a serious injury, a sidelined soccer star returns to his hometown and reluctantly agrees to train the local youth.
Release_year: 2016



In [25]:
# Simple predicate based on the partition key (unfortunatelly I only have one partition as of now)
def vector_search_filterordered(query, releaseYear, num_results=3, printQuery=False):
    query_embedding = generate_embeddings(query)

    querystring = "SELECT TOP {} c.id, c.type, c.title, c.rating, c.release_year, c.description, VectorDistance(c.docVector,{}) AS similarityScore FROM c WHERE c.partKey = '{}' ORDER BY VectorDistance(c.docVector,{})".format(num_results, query_embedding, releaseYear, query_embedding)
    
    if printQuery:
        print(querystring)

    results = container_client.query_items(
            query=querystring,
            enable_cross_partition_query=True)
    
    return results

In [30]:
# query = "What are some NoSQL databases in Azure?"
# query = "What are the services for event messaging patterns?"

#query = "Best romantic comedy movies to watch in family with kids coming to tenage years"
#query = "Action movie with zombies and a lot of blood"
query = "Sports movie that shows the struggle of a team to win the championship"

results = vector_search_filterordered(query, 2016, printQuery=True)
#results = vector_search_filterordered(query, 2011, printQuery=True)

for result in results: 
    # print(result)
    print(f"Similarity Score: {result['similarityScore']}")
    print(f"Id: {result['id']}")  
    print(f"Type: {result['type']}")        
    print(f"Title: {result['title']}")  
    print(f"Rating: {result['rating']}")  
    print(f"Description: {result['description']}") 
    print(f"Release_year: {result['release_year']}\n") 

# print("Consumed RUs: {} ".format(container_client.client_connection.last_response_headers['x-ms-request-charge']))

SELECT TOP 3 c.id, c.type, c.title, c.rating, c.release_year, c.description, VectorDistance(c.docVector,[-0.01079730037599802, -0.03456145152449608, -0.005688764620572329, -0.008892635814845562, -0.01865561492741108, 0.016019359230995178, 0.003147110342979431, -0.01974039152264595, -0.02428131178021431, -0.02835552580654621, 0.00843854434788227, 0.009964797645807266, 0.01687708869576454, 0.00899985246360302, 0.013912876136600971, -0.02709415927529335, 0.0313323512673378, -0.019765619188547134, -0.008350248448550701, -0.028204161673784256, -0.006294220685958862, -0.021430622786283493, 0.00019462495401967317, -0.005222058854997158, 0.01175593864172697, -0.011648722924292088, 0.01811322756111622, -0.025883248075842857, 0.02714461460709572, -0.02913757413625717, 0.03438486158847809, 0.0019519651541486382, -0.006007259711623192, -0.02272983081638813, -0.02565620094537735, -0.03509122505784035, -0.009125988930463791, -0.01420299056917429, 0.025441769510507584, -0.010721618309617043, 0.018365